# setup

In [1]:
import torch
import os
import matplotlib.pyplot as plt

from utils import H_imagenet, train, eval, model_select

from tqdm import tqdm

In [2]:
BATCH_SIZE = 128
EPOCHS = 10

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

# data

In [4]:
trn = H_imagenet('train', 'rn34')
val = H_imagenet('val', 'rn34')

Resolving data files:   0%|          | 0/52 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/52 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/39 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/52 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/52 [00:00<?, ?it/s]

In [5]:
trn_loader = torch.utils.data.DataLoader(trn, batch_size = BATCH_SIZE, shuffle = True)
val_loader = torch.utils.data.DataLoader(val, batch_size = BATCH_SIZE)

trn.__len__() / BATCH_SIZE, val.__len__() / BATCH_SIZE

(10009.1171875, 390.625)

# model

In [2]:
model_label = 'rn18'
model = model_select(model_label)

Using cache found in /home/josegfer/.cache/torch/hub/pytorch_vision_v0.10.0


In [5]:
model = model.to(device)

In [6]:
backbone = list(model.children())[:-1]
model = torch.nn.Sequential(*backbone)

# train

In [9]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())

In [10]:
log = []
for epoch in (range(EPOCHS)):
    loss_trn = train(model, trn_loader, optimizer, criterion, device)
    loss_val = eval(model, val_loader, criterion, device)
    log.append([loss_trn, loss_val])

  0%|          | 0/10010 [00:00<?, ?it/s]/home/josegfer/miniconda3/envs/mirror/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
  6%|▌         | 567/10010 [14:36<4:03:21,  1.55s/it]


KeyboardInterrupt: 

In [ ]:
plt.plot(log);

# write

In [7]:
if not os.path.exists('output'):
    os.makedirs('output')

torch.save(model.cpu(), 'output/rn18rn34.pt')